In [67]:
import numpy as np
import matplotlib.pyplot as plt

# 假设的目标函数
def calculate_core_loss(x):
    # 计算磁芯损耗的示例函数
    p = x['T'] + x['f'] + x['W'] + x['B_m'] + x['M']
    P = np.array(p)
    return P  # 这里用平方和作为损耗的示例

def calculate_energy_transfer(x):
    e = x['f'] * x['B_m']
    E = np.array(e)
    return E  # 这里用负的平方和作为能量传输的示例

# 目标函数
def objective_functions(x):
    P = calculate_core_loss(x)  # 磁芯损耗
    E = calculate_energy_transfer(x)  # 传输磁能
    return P, -E  # 返回损耗和负的能量




In [ ]:
#objectives = np.array([objective_functions(x) for x in population]) 2.7 3.6

In [68]:
P, E= objective_functions(df_g)

In [69]:
P

array([50057.01461734, 50051.01524825, 50052.01435666, 50049.01952671,
       50059.02290178])

In [65]:
p = np.array(P)

In [66]:
p

array([50057.01461734, 50051.01524825, 50052.01435666, 50049.01952671,
       50059.02290178])

In [52]:
E = 0 - E

In [53]:
E

0     731.305570
1     762.717515
2     718.120333
3     976.725834
4    1145.775853
dtype: float64

In [42]:
# 非支配排序
def non_dominated_sort(population):
    num_individuals = len(population)
    fronts = [[]]
    domination_counts = np.zeros(num_individuals)
    dominated_solutions = [[] for _ in range(num_individuals)]
    
    for p in range(num_individuals):
        for q in range(num_individuals):
            if p != q:
                if (population[p][0] <= population[q][0] and population[p][1] < population[q][1]) or \
                   (population[p][0] < population[q][0] and population[p][1] <= population[q][1]):
                    dominated_solutions[p].append(q)
                elif (population[q][0] <= population[p][0] and population[q][1] < population[p][1]) or \
                     (population[q][0] < population[p][0] and population[q][1] <= population[p][1]):
                    domination_counts[p] += 1
                    
        if domination_counts[p] == 0:
            fronts[0].append(p)
    
    current_front = 0
    while len(fronts[current_front]) > 0:
        next_front = []
        for p in fronts[current_front]:
            for q in dominated_solutions[p]:
                domination_counts[q] -= 1
                if domination_counts[q] == 0:
                    next_front.append(q)
        current_front += 1
        fronts.append(next_front)
    
    return fronts[:-1]

In [85]:
non_dominated_sort(objectives)

[[3, 4], [1], [0, 2]]

In [43]:
# 生成初始种群
def generate_population(size, num_variables):
    return np.random.rand(size, num_variables) * 5  # 假设变量范围是0到5

# 选择操作
def selection(population, fronts):
    selected = []
    for front in fronts:
        if len(selected) + len(front) > len(population):
            break
        selected.extend(front)
    return selected

In [44]:
# 交叉和变异
def crossover_and_mutate(population):
    np.random.shuffle(population)
    new_population = []
    for i in range(0, len(population), 2):
        if i + 1 < len(population):
            child1 = (population[i] + population[i + 1]) / 2
            child2 = (population[i] + population[i + 1]) / 2 + np.random.normal(0, 0.1, population[i].shape)
            new_population.append(child1)
            new_population.append(child2)
    return np.clip(np.array(new_population), 0, 5)  # 保证变量在范围内


In [46]:
# NSGA-II主循环
def nsga_ii(num_generations, population_size, num_variables):
    population = generate_population(population_size, num_variables)
    for _ in range(num_generations):
        objectives = np.array([objective_functions(x) for x in population])
        fronts = non_dominated_sort(objectives)
        selected_indices = selection(population, fronts)
        population = crossover_and_mutate(population[selected_indices])
    
    return population

In [57]:
population = generate_population(population_size, num_variables)

In [59]:
population

array([[2.73874536, 4.91036253],
       [3.74354445, 3.05207984],
       [1.66388464, 0.81525921],
       [1.77571132, 4.69256191],
       [3.43454328, 3.90177367],
       [1.65247104, 2.26505023],
       [3.81740611, 3.77756193],
       [4.40770309, 1.45547279],
       [1.24878264, 3.87731311],
       [2.16068746, 0.35146675],
       [1.61128098, 4.92879292],
       [0.12525635, 4.66192027],
       [3.68762356, 0.25667103],
       [0.4461662 , 1.868183  ],
       [3.73989074, 4.52936832],
       [2.51885429, 0.644051  ],
       [2.54360032, 3.71680713],
       [0.0403196 , 1.87608066],
       [3.24353096, 2.77737769],
       [2.01316771, 2.24932876],
       [1.13580766, 3.73686191],
       [1.21215043, 2.20304258],
       [1.42215251, 0.69912817],
       [1.56343665, 3.71862118],
       [2.12902627, 1.63522939],
       [1.98140699, 1.03748528],
       [2.55993544, 2.78295723],
       [1.92519744, 1.8426099 ],
       [4.12635226, 3.89685823],
       [1.94589375, 0.24820218],
       [1.

In [81]:
[objective_functions(x) for x in df_g]

TypeError: string indices must be integers

In [82]:
objectives = np.array([objective_functions(row) for _, row in df_g.iterrows()])

In [83]:
objectives

array([[50057.01461734,  -731.30557023],
       [50051.01524825,  -762.71751502],
       [50052.01435666,  -718.12033328],
       [50049.01952671,  -976.72583412],
       [50059.02290178, -1145.77585328]])

In [77]:
objectives = np.array([objective_functions(x) for x in df_g])

TypeError: string indices must be integers

In [54]:
# 运行NSGA-II
x = df_g
num_generations = 100
population_size = 50
num_variables = 2
final_population = nsga_ii(num_generations, population_size, num_variables)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [75]:
[x **2 for x in range(4)]

[0, 1, 4, 9]

In [55]:
# 可视化结果
P_values = [calculate_core_loss(x) for x in final_population]
E_values = [calculate_energy_transfer(x) for x in final_population]
plt.scatter(P_values, -np.array(E_values) + 10)  # 将能量值调整为正值以便可视化
plt.xlabel("Magnetic Core Loss (P)")
plt.ylabel("Energy Transfer (E)")
plt.title("NSGA-II Result for Magnetic Component Optimization")
plt.grid()
plt.show()

NameError: name 'final_population' is not defined

In [16]:
import pandas as pd

# 读取 Excel 文件中的一个工作表
df = pd.read_excel('C:/Users/lenovo/Desktop/训练集.xlsx', sheet_name='材料1')  # 指定工作表名

In [17]:
import numpy as py
df.rename(columns={df.columns[0]: 'T'}, inplace=True)
df.rename(columns={df.columns[1]: 'f'}, inplace=True)
df.rename(columns={df.columns[3]: 'W'}, inplace=True)

In [18]:
df=df.head()

In [19]:
df_g = df

In [20]:
df_g['W']=[1,3,3,2,1]

In [21]:
B_m = [0.014617341,0.015248251,0.014356664,0.019526706,0.022901776]

In [22]:
M = [1,3,4,2,3]

In [25]:
data = [df_g.iloc[:,0],df_g.iloc[:,1],df_g.iloc[:,3],B_m,M]

In [31]:
df_g['B_m'] = B_m
df_g['M'] = M

In [33]:
df_g

,T,f,磁芯损耗，w/m3,W,0（磁通密度B，T）,1,2,3,4,5,...,1016,1017,1018,1019,1020,1021,1022,1023,B_m,M
0,25,50030,1997.955250,1,0.000213,0.000389,0.000566,0.000743,0.000919,0.001096,...,-0.001198,-0.001022,-0.000846,-0.000669,-0.000492,-0.000316,-0.000140,0.000036,0.014617,1
1,25,50020,2427.749830,3,-0.000551,-0.000358,-0.000165,0.000028,0.000221,0.000413,...,-0.002091,-0.001899,-0.001707,-0.001515,-0.001322,-0.001129,-0.000937,-0.000744,0.015248,3
2,25,50020,3332.725760,3,-0.003780,-0.003564,-0.003349,-0.003134,-0.002919,-0.002704,...,-0.005499,-0.005285,-0.005070,-0.004855,-0.004640,-0.004425,-0.004210,-0.003995,0.014357,4
3,25,50020,4502.908007,2,-0.000511,-0.000267,-0.000023,0.000222,0.000466,0.000711,...,-0.002463,-0.002220,-0.001976,-0.001732,-0.001488,-0.001244,-0.001000,-0.000756,0.019527,2
4,25,50030,6063.023248,1,0.000458,0.000732,0.001007,0.001281,0.001555,0.001830,...,-0.001741,-0.001466,-0.001192,-0.000917,-0.000642,-0.000367,-0.000092,0.000183,0.022902,3


In [72]:
q = df_g['f'] * df_g['B_m']
p = df_g['T'] + df_g['f'] 

In [73]:
p

0    50055
1    50045
2    50045
3    50045
4    50055
dtype: int64

In [74]:
Q = np.array(q)